## Библиотеки

Сегодня нам понадобятся библиотеки

- `pandas` - работа с данными
- `import numpy as np` - полезные функции (кстати, `pandas` внутри использует много `numpy`)
- `DecisionTreeClassifier` - дерево решений 
- `sklearn.model_selection` - проверка модели
- `xgboost` - более продвинутая модель  
- `eli5` - интерпретирование моделей, чтобы увидеть важность этих функций
- `from collections import Counter` это будет полезно для удобного подсчета количества повторов (например, сколько раз покупатель покупал продукт x).
- `gc` - сборщик "мусора"

In [ ]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import eli5 
from collections import Counter

import gc

## Загрузим данные

In [ ]:
df = pd.read_hdf("../input/data.h5")

print(df.shape)
df.sample(5)

(820906, 9)


,order_id,customer_id,product_id,quantity,price_unit,price_total,country_id,order_date,is_canceled
110727,7794,165,362,1,295,295,0,2011-04-17 12:49:00,False
800094,51865,2406,196,3,375,1125,0,2010-11-28 13:43:00,False
95785,6676,1914,152,2,850,1700,0,2011-03-31 11:58:00,True
628845,41391,1253,393,24,85,2040,0,2010-07-27 15:06:00,False
97044,6782,2190,240,3,125,375,0,2011-04-01 11:54:00,False


## Подготавливаем данные


Это код из предыдущего урока. Мы просто преобразуем код в функцию, чтобы нам было легче экспериментировать позже.

In [ ]:
df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)


df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc


top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())

del df_customers
gc.collect()

def feature_engineering(df):
    df_customers = (
        df
        .groupby("customer_id", as_index=False)
        .agg(
            count_orders=("order_id", lambda x: len(set(x))),
            count_unq_products=("product_id", lambda x: len(set(x))),
            sum_quantity=("quantity", np.sum),
            sum_price_unit=("price_unit", np.sum),
            sum_price_total=("price_total", np.sum),
            count_unq_countries=("country_id", lambda x: len(set(x))),
            prob_canceled=("is_canceled", np.mean)
        )
    )
    
    
    
    return df_customers


def get_feats(df_customers, black_list=["most_revenue_customer"]):
    feats = list(df_customers.select_dtypes([np.number, bool]).columns)
    return [x for x in feats if x not in black_list]

def get_X_y(df_customers, top_customers, feats):
    df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers)
    
    X = df_customers[feats].values
    y = df_customers["most_revenue_customer"].values
    
    return X, y


def train_and_get_scores(model, X, y, scoring="accuracy", cv=5):

    scores = cross_val_score(model, X, y, scoring=scoring, cv=cv)
    return np.mean(scores), np.std(scores)

## Модель 1

👇 Именно то, что мы сделали недавно.

In [ ]:
df_customers = feature_engineering(df)
feats = get_feats(df_customers)
X, y = get_X_y(df_customers, top_customers, feats)
model = DecisionTreeClassifier(max_depth=3)

train_and_get_scores(model, X, y)

(0.9996598639455783, 0.0004165798882284457)

У нас результат 99,9%. Это подозрительно хороший результат.

Можешь быть счастлив, но... на самом деле я расскажу Тебе об одном механизме размышления, который я стараюсь использовать регулярно. Я называю это маятником. Хотя думаю, что этот метод был выдуман давным давно (и возможно, даже имеет некоторое название), но это не так важно. Ведь в реальной жизни важнее уметь применять, чем делать вид "что все знаешь" и при этом используешь умные слова, но понятия не имеешь - как это действительно работает. Согласен(а)?

Идея состоит в том, чтобы сначала подходить оптимистично и даже можно сильно упрощать реальность. Затем, когда у нас есть хороший результат, мы «переобуваемся». Обычно это звучит в негативном ключе, но здесь смысл в другом. Посмотреть на ситуацию с другой стороны. Мы играем роль активного «критика», который пытается сделать все, чтобы доказать бессмысленность того, что мы сами же и сделали. И переключение от оптимиста в сторону критика повторяем снова и снова. Делаем несколько итераций :). 
Это порой утомляющий процесс, когда выкладываешься на 100% и на оба фронта. Часто требуются даже разные компетенции - именно поэтому хорошо, когда в одной команде есть люди с одного и со второго лагеря.

Но самое главное, что, переключаясь между ролями, можно очень быстро развиваться и прощупывать возможности, а также минимизировать риски. В результате потихоньку, но решительно двигаешься вперед!

## 🧠 Включим критическое мышление

На данный момент мы сделали первый шаг, который является оптимистичным, и получили результат 99%. Очень хорошо, но ... Проблема в том, что `price_unit_total` слишком сильная характеристика. Таким образом, эта функция по сути напрямую сообщает, принадлежит ли покупатель к сегменту `top_customers` или нет. Как именно? Давай вспомним, что значит эта цифра? Это общее количество денег, которые потратил клиент в нашем интернет магазине (который анализируем). Существует некоторая граница, пересекая которую, начинаешь принадлежать к сегменту `top_customers`. Говоря простым языком, каждый, кто потратил миллион (или даже гораздо меньше) становится "любимчиком" среди клиентов. Соответственно, наш алгоритм (даже относительно простой, как дерево решений) найдет это автоматически. Не веришь? Давай проверять.


Давай быстро посмотрим что там внутри - для этого используем `eli5` (eli5 - это сокращение от `Explain Like I'm 5` т.е. объясни мне как пятилетнему ребенку, что подразумевает, что объяснение должно быть очень простым).

In [ ]:
model = DecisionTreeClassifier(max_depth=3)
model.fit(X, y)
eli5.show_weights(model, feature_names=feats)

Weight,Feature
1.0000,sum_price_total
0,prob_canceled
0,count_unq_countries
0,sum_price_unit
0,sum_quantity
0,count_unq_products
0,count_orders
0,customer_id


## Что это значит и как это интерпретировать?

Сразу видно таблицу в двумя колонками:
- Weight - вес важности признака
- Feature - название признака

Видим, что `sum_price_total` по сути сделал "всю работу" и модель быстро "уловила" важность этого признака. Что значит "уловила"? Чуть ниже (под таблицей) есть дерево решений (три прямоугольника и две стрелки, видишь? 👀). 

В корне дерева (т.е. самый верхний прямоугольник,  кстати, здесь наоборот - корень дерева вверху 😉) видно 4 строки:
- `sum_price_total <= 263595.0`
- `gini = 0.349`
- `samples = 100.0%`
- `value = [0.775, 0.225]`

Самое интересное - это первая строка `sum_price_total <= 263595.0` - это есть условие. Дословно это значит, что если клиент потратил **£2635,95** тогда он присоединяется в клуб `most_revenue_customer`. Благодаря этому условию, задача решается при помощи одного "удара" (разделения). Но это задача достаточно простая, поэтому алгоритм быстро нашел решение.


Кстати, если интересно, то можно немного усложнить жизнь для алгоритма и удалить признак `"sum_price_total"`, а точнее говоря - спрятать этот признак, чтобы его не было в X. Давай быстро проведем эксперимент.

In [ ]:
black_list = ["most_revenue_customer", "sum_price_total"]
feats = [x for x in feats if x not in black_list]

X = df_customers[feats].values
y = df_customers["most_revenue_customer"].values


model = DecisionTreeClassifier(max_depth=3)
model.fit(X, y)
eli5.show_weights(model, feature_names=feats)

Weight,Feature
0.9174,sum_quantity
0.0426,count_orders
0.0400,sum_price_unit
0,prob_canceled
0,count_unq_countries
0,count_unq_products
0,customer_id


Теперь видно более "ветвистое" и сложное дерево. Также видно, что сейчас доминирует признак `sum_quantity`, который считает количество штук среди всех заказов для каждого клиента.

Можем даже немного вместе проследить, как появились условия:
- `sum_quantity <= 1534.5`
- `count_orders <= 10.5`
- `um_price_unit <= 31075.5`
- ...


Кстати, а как "просело" качество модели, когда спрятали признак `sum_price_total`?

In [ ]:
df_customers = feature_engineering(df)
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)
model = DecisionTreeClassifier(max_depth=3)

train_and_get_scores(model, X, y)

(0.7863585178752353, 0.26358237322275296)

Как видишь, качество модели упало до 78% (было 99%). С одной стороны, грустно 😂, но с другой стороны, мы используем модель машинного обучения, чтобы найти менее очевидные зависимости. Потому что очевидные вещи легко найти самостоятельно на пальцах или калькуляторе 🤦‍♂️.


### Оптимистичное мышление
А теперь вернемся к оптимисту. Ты помнишь, в чем принцип маятника? Мы сомневаемся или наоборот - полны оптимизма, и так вперед и назад - по кругу;)

Сейчас мы перейдем на более трудную модель и это должно улучшить качество модели (улавливать более сложные взаимосвязи). Давай использовать алгоритмы из семейства "бустинг" - а именно `xgboost`. Это название может тебе ни о чем не говорить, и на данном этапе - это нормально 👌. Просто запомни, что - это проверенный в бою алгоритм, который также ведет себя уверенно и на "продакшене" (анг. `production`).

## XGBoost
Обрати внимание, что, по сравнению с предыдущим кодом, изменилась только эта строка: `model = xgb.XGBClassifier(**xgb_params)` и добавили параметры для нашей модели в пару строк (чтобы не делать одну длинную т.к. это трудно читается).

In [ ]:
df_customers = feature_engineering(df)
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)

xgb_params = dict(
    max_depth=5, 
    n_estimators=50, 
    learning_rate=0.3, 
    use_label_encoder=False, 
    objective="binary:logistic", 
    eval_metric="error",
    random_state=0)
model = xgb.XGBClassifier(**xgb_params)

train_and_get_scores(model, X, y)

(0.8106769431176726, 0.08696513375768444)

## У нас результат 81%.

Попробуем сделать первую интерпретацию модели и посмотрим, что она считает важным.

In [ ]:
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.3, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=4,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [ ]:
eli5.show_weights(model, feature_names=feats)

Weight,Feature
0.6835,sum_quantity
0.1309,count_orders
0.0776,sum_price_unit
0.0545,count_unq_products
0.0298,prob_canceled
0.0235,customer_id
0,count_unq_countries


В данном случае у нас уже не одно дерево внутри, а целых 50 штук (за это отвечает параметр `n_estimators=50`).  Кстати, может быть гораздо больше: 100 или даже больше 1000, но пока что нам это не нужно. Давай сделаем первые наблюдения и выводы.

### Предварительные выводы:
#### TOP3 
- `sum_quantity` звучит как самая **важная** особенность* (осторожно, ниже комментарий к звездочке)
- `count_orders`
- `sum_price_unit`

Похоже, что функция `count_unq_countries` мало интересна.


☝️ * Нужно быть осторожным с тем, что является «важным», потому что опыт показывает, что это может быть довольно изменчивым, и нужно научиться ориентироваться в этой изменчивости. Если бы все было так просто, то этот процесс уже давно был бы на 100% автоматизирован. 


## Следующий шаг

Теперь мы можем немного порасcуждать над этими признаками, но это не имеет большого значения. Эти особенности сами по себе очевидны. Даже если мы сделаем правильные выводы (что не факт), то какая будет ценность? Нужно все-таки использовать машинное обучение (желательно находить менее очевидные зависимости, которые вызывают восторг и удивление!)

Давай сгенерируем больше признаков и пусть модель сама найдет то, на что стоит обратить внимание :)

## Продукты
Мы можем добавить отдельный столбец для каждого продукта и добавить значения в этот столбец -  сколько продуктов уже было куплено (то есть сумму, например, Саша купил 5 упаковок молока). Таким образом, мы хотим выяснить, какие продукты «самые интересные», на которые стоит обратить внимание.

☝️ Конечно, мы можем проверить каждый продукт (товар) вручную, но помни, что у нас их более 3.8 тыс. Делать проверку вручную - это  очень утомительно, занимает кучу времени да еще и дорогим процессом является (зарплаты нужно добавить и т.д). Давай лучше думать по принципу 80/20 и использовать правильные решения, например, машинное обучение, оно будет быстрее, надежнее и эффективнее.

Только сначала нам нужно подготовить признаки. И желательно, сделать их гораздо больше (чтобы было где "разгуляться" алгоритму). Давай изменим нашу функцию `feature_engineering`. 

In [ ]:
def feature_engineering(df):
    
    def counter(vals):
        cntr = Counter()
        cntr.update(vals)
        return cntr
    
    df_customers = (
        df
        .groupby("customer_id", as_index=False)
        .agg(
            count_orders=("order_id", lambda x: len(set(x))),
            count_unq_products=("product_id", lambda x: len(set(x))),
            count_by_products=("product_id", lambda x:  counter(x) ),
            sum_quantity=("quantity", np.sum),
            sum_price_unit=("price_unit", np.sum),
            sum_price_total=("price_total", np.sum),
            count_unq_countries=("country_id", lambda x: len(set(x))),
            prob_canceled=("is_canceled", np.mean)
        )
    )
    
    
    
    return df_customers


df_customers  = feature_engineering(df)
df_customers.head()

,customer_id,count_orders,count_unq_products,count_by_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled
0,0,159,93,"{0: 89, 1: 73, 2: 69, 3: 68, 4: 32, 5: 36, 6: ...",22976,895827,5746671,1,0.010966
1,1,40,184,"{8: 8, 9: 4, 10: 4, 11: 4, 12: 3, 13: 15, 14: ...",3514,205962,910113,1,0.097727
2,2,30,139,"{25: 9, 26: 7, 27: 7, 28: 3, 29: 4, 30: 10, 31...",8971,95500,1337464,1,0.009050
3,3,9,40,"{44: 7, 1781: 1, 1179: 1, 112: 1, 508: 3, 581:...",1108,18626,226381,1,0.000000
4,4,14,11,"{54: 14, 1496: 2, 2288: 1, 2315: 1, 3432: 1, 4...",390,20028,346978,1,0.307692


Появился новый столбец `count_by_products`. Внутри есть словарь. Посмотрим внимательнее на этот словарь 👀.

In [ ]:
df_customers["count_by_products"]

0       {0: 89, 1: 73, 2: 69, 3: 68, 4: 32, 5: 36, 6: ...
1       {8: 8, 9: 4, 10: 4, 11: 4, 12: 3, 13: 15, 14: ...
2       {25: 9, 26: 7, 27: 7, 28: 3, 29: 4, 30: 10, 31...
3       {44: 7, 1781: 1, 1179: 1, 112: 1, 508: 3, 581:...
4       {54: 14, 1496: 2, 2288: 1, 2315: 1, 3432: 1, 4...
                              ...                        
5874    {1709: 1, 409: 1, 170: 1, 662: 1, 253: 1, 255:...
5875    {2368: 1, 300: 1, 1385: 1, 2412: 1, 2059: 1, 1...
5876    {1827: 1, 869: 1, 2157: 1, 335: 1, 1931: 1, 21...
5877    {1038: 1, 701: 2, 495: 1, 353: 1, 189: 1, 484:...
5878                                             {694: 2}
Name: count_by_products, Length: 5879, dtype: object

Давай для примера возьмем первую строку и прочитаем вместе - что там. На практике это означает, что `customer_id=0` купил `product_id= 0` - 89 раз, `product_id=1` - 73 раза,` product_id=3` - 68 раз и т.д.


Было бы неплохо, если бы мы распаковали словарь на отдельные столбцы, благо сделать это очень просто, используя `pandas` и пару простых трюков. ⌛️ Но нужно немного подождать пока сервер будет работать.

In [ ]:
df_count_products = df_customers["count_by_products"].apply(pd.Series).fillna(-1)
df_count_products.columns = ["product_{}".format(x) for x in df_count_products.columns]

df_count_products.head(5)

,product_0,product_1,product_2,product_3,product_4,product_5,product_6,product_7,product_8,product_9,...,product_4216,product_4219,product_4220,product_4222,product_4226,product_4231,product_4236,product_4237,product_4238,product_4239
0,89.0,73.0,69.0,68.0,32.0,36.0,34.0,35.0,18.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,6.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8.0,4.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


Теперь у нас есть отдельный `dataframe` с 3878 столбцами. Каждый столбец - это `product_id`. Значение -1 означает отсутствие информации, т.е. покупатель не покупал этот товар, а любое другое число (больше 0) означает -  сколько раз этот товар был куплен (это немного оптимистичное мнение, но пока мы можем предположить, что это в порядке).

☝️ Обрати внимание, что появилась строка для переименования продуктов. `df_count_products.columns = ["product _ {". format(x) for x in df_count_products.columns]` Здесь мы делаем очень простые вещи, вместо простого идентификатора, например 0 или 2, мы добавляем префикс для столбца `product_0` или `product_2`. Это нам сейчас пригодится.

### Следующий шаг
Давай объединим новую табличку `df_count_products` с таблицей `df_customers` и на всех этих данных обучим модель. Чтобы объединить это, мы будем использовать функцию `concat()`. 

Объясняю на пальцах, что мы хотим сделать. Имеется:
- таблица 1 `df_customers.shape` => `(5879, 9)` т.е. 5879 строк и 9 столбцов
- таблица 2 `df_count_products.shape` => `(5879, 3878)` т.е. 5879 строк и 3878 столбцов

Как видишь, количество строк одинаковое. И это, конечно, не случайно, ведь это наше количество клиентов. То, что появилось в таблице 2 - это дополнительная информация для каждого клиента. Теперь нам нужно объединить в одну таблицу все столбцы (признаки) о клиенте, т.е. в результате объединения получается `9+3878=3887` столбцов.

Можно даже так сказать, берем две таблицы и прикладываем их друг к другу, как две книги. Другими словами, количество строк останется прежним, количество столбцов увеличится на 3878 😱. Алгоритму это обязательно "понравится"😉 (конечно, у модели нет собственного мнения, но если бы оно было - то скорее всего - оно было бы такое, либо нам хотелось бы, чтобы оно такое было 🤫), т.к. будет больше признаков и можно будет найти что-то интересное, ну или по крайней мере - будет что оптимизировать.

Хватит слов, объединяем!

In [ ]:
df_customers_big = pd.concat([df_customers, df_count_products], axis=1)
df_customers_big.shape

(5879, 3887)

Как и ожидалось, количество строк осталось прежним - 5879 (поскольку количество строк - это клиенты, а новых клиентов не было). Количество столбцов - увеличилось до 3887. Теперь можно сказать, что для каждого покупателя у нас теперь 3887 признаков (характеристик).

## 🤖 Время обучать модель

Для начала проверим, улучшилось ли для нас качество модели.

⌛️⌛️⌛️ Внимание! Данных стало больше (было 9 признаков, а стало 3887), комбинаций стало гораздо больше, поэтому алгоритму нужно время на тренировку модели.

Алгоритм выполняет «простую» и повторяющуюся работу, в то время, как Ты можешь тихонько в стороне мыслить дальше, о чем-то более концептуальном 😉. Привыкай - это будущее, которое к нам уже пришло :).

In [ ]:
feats = get_feats(df_customers_big, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers_big, top_customers, feats)

xgb_params = dict(
    max_depth=5, 
    n_estimators=50, 
    learning_rate=0.3, 
    use_label_encoder=False, 
    objective="binary:logistic", 
    eval_metric="error",
    random_state=0)
model = xgb.XGBClassifier(**xgb_params)

train_and_get_scores(model, X, y)

(0.8474117817339701, 0.08924212099402075)

Качество модели улучшилось, у нас уже ~ 85%.

Напомню, что только деревья решений дали нам 77%. XGBoost по базовым характеристикам дал нам точность 81% и теперь, добавив новых признаков, улучшили точность модели до ~ 85%.

Может показаться, что это не много, потому что у нас было 99,9%. Хочу снова такого результата! 😂

Возможно, это плохие новости, но часто модель «останавливается» на отметке 60-70% и тогда приходится очень стараться выжать больше. Ну разве что мы решаем очевидные задачи, но тогда не очень интересно применение машинного обучения (трудно объяснить выгоду). Это как за булочками ездить на Боинге - немного неудобно (хотя летать между континентами - очень даже хорошо). 

Теперь давай обучим еще раз модель, чтобы у нас был доступ к ней напрямую. Выше мы использовали дополнительные функции, которые удобны, но сама модель "теряется" где-то в недрах эксперимента и до нас доходит только информация, например, о качестве (это тоже порой удобно).

Тренируем модель с нуля, чтобы у нас был доступ напрямую к ее состоянию (т.е. конечное решение).

In [ ]:
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.3, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=50, n_jobs=4,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

Теперь пришло время посмотреть на важность признаков.

In [ ]:
eli5.show_weights(model, feature_names=feats, top=50)

Weight,Feature
0.0992,sum_quantity
0.0179,product_545
0.0168,product_99
0.0145,count_orders
0.0132,product_1777
0.0131,product_847
0.0124,product_1280
0.0120,product_209
0.0103,product_1604
0.0101,product_2908


Что ж, теперь хоть есть на что посмотреть :). Ты можешь увидеть, что важность признаков `product_545` и` product_99` превышает даже важность `count_orders`. Конечно, как я уже сказал, здесь нужно делать выводы осторожно. Однако однозначно то, что `product_id=545` заслуживает нашего внимания. 🥳 Ну что ж - продукт 545 - держись, мы идем тебя проверять :).

In [ ]:
customer_ids_by_product = set(df[ df["product_id"] == 545 ]["customer_id"].unique())
len(customer_ids_by_product )

168

Этот товар купили не так много клиентов, всего 168. Посмотрим, сколько из них относятся к сегменту `most_revenue_customer`.

In [ ]:
df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers) 

In [ ]:
df_customers[ df_customers.customer_id.isin(customer_ids_by_product) ]["most_revenue_customer"].mean()

0.4166666666666667

Итак, мы видим, что у нас есть 42% клиентов, которые купили по крайней мере `product_id=545` и принадлежат к сегменту `most_revenue_customer`.

Напомню, что у нас обычно 22% клиентов относятся к сегменту `most_revenue_customer`.

In [ ]:
df_customers["most_revenue_customer"].mean()

0.225208368770199

Мы можем быстро выдвинуть гипотезу (для проверки) - если покупатель купил `product_id=545`, вероятность того, что он будет принадлежать сегменту `most_revenue_customer`, почти удвоится (с 23% до 42%). Это звучит как "aha moment". Эта гипотеза заслуживает дальнейшего изучения, но если она, действительно, работает, это означает, что стоит просто побудить людей больше покупать `product_id=545`.

#### Давай узнаем, сколько стоит этот товар. Может это какая-то роскошь ⚜️.

In [ ]:
df[ df.product_id == 545 ]["price_unit"].value_counts()

125    126
39     103
42       5
246      3
106      1
Name: price_unit, dtype: int64

Цена на этот товар варьируется, но чаще всего цена за единицу составляет всего 1,25 или 3,9. Такой интересный вывод был сделан так легко.

## Задача - сделать аналогичный анализ для продукта.

### 💡 План действий: 

1. Подготовить `df_products`
2. Добавить больше признаков. Прежде мы добавили `product_id`, но Ты можешь сделать симметричным или придумать что-то еще (возможно ,` order_id`) или что-то еще, идей много :)
3. Обучить модель.

In [ ]:
df_products = (
    df[ ["price_total", "product_id"] ]
    .groupby("product_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "product_price_total"})
)


df_products["cumsum"] = df_products["product_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_products["most_revenue_product"] = df_products["cumsum"] < value_80prc


top_products = set(df_products[ df_products["most_revenue_product"] ]["product_id"].unique())

del df_products
gc.collect()

def feature_engineering(df):

    def counter(vals):
        cntr = Counter()
        cntr.update(vals)
        return cntr
    
    df_products = (
        df
        .groupby("product_id", as_index=False)
        .agg(
            count_orders=("order_id", lambda x: len(set(x))),
            count_unq_products=("customer_id", lambda x: len(set(x))),
            sum_quantity=("quantity", np.sum),
            sum_price_unit=("price_unit", np.sum),
            sum_price_total=("price_total", np.sum),
            count_by_customer=("customer_id", lambda x: counter(x)),
            count_unq_countries=("country_id", lambda x: len(set(x))),
            prob_canceled=("is_canceled", np.mean)
        )
    )
    
    return df_products

def get_feats(df_products, black_list=["most_revenue_product", "sum_price_total"]):
    feats = list(df_products.select_dtypes([np.number, bool]).columns)
    return [x for x in feats if x not in black_list]
    
def get_X_y(df_product, top_products, feats):
    df_products["most_revenue_product"] = df_products["product_id"].map(lambda x: x in top_products)
    
    X = df_products_big[feats].values
    y = df_products_big["most_revenue_product"].values
    
    return X, y

def train_and_get_scores(model, X, y, scoring="accuracy", cv=5):

    scores = cross_val_score(model, X, y, scoring="accuracy", cv=cv)
    return np.mean(scores), np.std(scores)

df_products = feature_engineering(df)

df_count_customer = df_products["count_by_customers"].apply(pd.Series).fillna(-1)
df_count_customer.columns = ["customer_{}".format(x) for x in df_count_customer.columns]

df_products_big = pd.concat([df_products, df_count_customers], axis = 1)
black_list = ["most_revenue_product", 'sum_price_total']
feats = get_feats(df_products_big, black_list)

X, y = get_X_y(df_product_big, top_products, feats)

xgb_params = dict(
    max_depth=5, 
    n_estimators=50, 
    learning_rate=0.3, 
    use_label_encoder=False, 
    objective="binary:logistic", 
    eval_metric="error",
    random_state=0)

model = xgb.XGBClassifier(**xgb_params)

train_and_get_scores(model, X, y)

## 🧠 Включим критическое мышление


Весь наш анализ был основан на `price_total`, что является доходом. А как насчет возвратов? Может, пора было еще немного углубиться в кроличью нору?

На что еще стоит обратить внимание? Сразу скажу, что есть еще много чего;)

## 😇 Включим творческое мышление

Подумай, как можно использовать полученные здесь знания?

В идеале есть примеры с Твоего собственного двора. Если Ты этого не сделаешь, думая, что Ты просто не сможешь этого сделать - подумай о своем окружении.

Дело не только в E-commerce. Это может быть любая отрасль. Правило 80/20 универсально. То же самое и с «aha-moment».